In [1]:
import pandas as pd

Считаем данные с диска

In [2]:
good = pd.read_csv('good.tsv', sep = '\t')

Покажем три случайные строки текст - ответ

In [3]:
good.sample(3)

,context_2,context_1,context_0,reply,label
656,NaN,это и правда хорошо .,ладно .,твоё здоровье .,good
5502,один миллион жителей против 81 заложника .,а за всем этим стоит фрэнк хаммель .,"из - за того , что мы проигнорировали , отказа...",". . которые погибли , делая то , что они счита...",good
11203,NaN,видео с вами есть по всему интернету !,парень - шиншилла !,супер !,good


Выбираем случайный из всех ответов на данный вопрос

отфильтровываем по вопросу реплики

In [4]:
rep = good[good.context_0 == 'как дела ?'].reply

In [5]:
rep

25084               пока без изменений .
25085               нормально . у тебя ?
25086                        нормально .
25087               нормально , хорошо .
25088                  нормально , мам .
25089                       потихоньку .
39809    хорошо , спасибо . как у тебя ?
39810                     лучше некуда .
39811             нормально , а у тебя ?
39812                хорошо . как ваши ?
39813                          отлично .
39814        хорошо . а что ты делаешь ?
42086                хорошо . как ваши ?
42087                 хорошо . а у вас ?
42088                         так себе .
42089                       все хорошо .
42090              все хорошо , хорошо .
42091            нормально , нормально .
43270                    как сажа бела .
43271                       дела супер .
43272                        нормально .
43273                     дела отлично .
43274                       потихоньку .
43275                          все гуд .
55580           

Убедимся, что есть хотя бы один ответ, выбираем один случайный и печатаем

In [6]:
if rep.shape[0] > 0:
    print(rep.sample(1).iloc[0])

как сажа бела .


Превратим тексты в числовые векторы

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

Создаем объект, который будет преобразовывать текст в числовые векторы

In [8]:
vectorizer = TfidfVectorizer()

Обучаем его на всех контекстах

In [9]:
vectorizer.fit(good.context_0)

TfidfVectorizer()

Записываем в матрицу сколько раз каждое слово встречалось в каждом тексте

In [10]:
matrix_big = vectorizer.transform(good.context_0)

In [11]:
print ("Размер матрицы: ")
print (matrix_big.shape)

Размер матрицы: 
(60049, 14123)


Сокращаем размерность с 14123 до 300

Импортируем метод, известный как Метод главных компонент

In [12]:
from sklearn.decomposition import TruncatedSVD

In [13]:
svd = TruncatedSVD(n_components=300)
svd.fit(matrix_big)

TruncatedSVD(n_components=300)

Получим трансформированную матрицу

In [15]:
matrix_small = svd.transform(matrix_big)

In [16]:
print(matrix_small.shape)

(60049, 300)


In [17]:
print ("Коэффициент уменьшения матрицы: ")
print ( svd.explained_variance_ratio_.sum())

Коэффициент уменьшения матрицы: 
0.43950115167888254


Пишем класс для случайного выбора среди подходящих ответов

In [18]:
from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator
import numpy as np

Функция для создания вероятностного распределения

In [19]:
def softmax(x):
    proba = np.exp(-x)
    return proba / sum(proba)

Класс для случайного выбора одного из случайных ближайших соседей

In [20]:
class NeighborSampler(BaseEstimator):
    def __init__(self, k=5, temperature=10.0):
        self.k=k
        self.temperature = temperature
    def fit(self, X, y):
        self.tree_ = BallTree(X)
        self.y_ = np.array(y)
    def predict(self, X, random_state=None):
        distances, indices = self.tree_.query(X, return_distance=True, k=self.k)
        result = []
        for distance, index in zip(distances, indices):
            result.append(np.random.choice(index, p=softmax(distance * self.temperature)))
        return self.y_[result]

Соединяем векторизацию, сокращение размерности и поиск соседей

In [21]:
from sklearn.pipeline import make_pipeline

In [22]:
ns = NeighborSampler()

answer_id - код ответа в массиве, который получается при поиске ближайшего ответа


In [23]:
ns.fit(matrix_small, good.reply) 
pipe = make_pipeline(vectorizer, svd, ns)

In [24]:
print(pipe.predict(['почему так происходит?']))

['потому что слишком много болтаю .']


In [25]:
print(pipe.predict(['есть ли жизнь на Марсе?']))

['нет , мы проверяли .']


In [26]:
import pymorphy2
import re

Настраиваем язык для библиотеки морфологии

In [27]:
morph = pymorphy2.MorphAnalyzer(lang='ru')

Для удобства общения

In [28]:
print("Пишите ваш вопрос, слова exit или выход для выхода")

request=""

while request not in ['exit', 'выход']:
    # получим текст от ввода
    request=input()

    # разберем фразу на слова
    words= re.split('\W',request)
    phrase=""
    for word in words:
        word = morph.parse(word)[0].normal_form  # морфируем слово вопроса в нормальную словоформу
    # Нормализуем словоформу каждого слова и соберем обратно фразу
    phrase = phrase + word + " "

    # запустим функцию и получим код ответа
    answer = pipe.predict([phrase.strip()])

    # выведем текст ответа
    print (answer)

Пишите ваш вопрос, слова exit или выход для выхода
Кто ты
['что я ?']
Что ты?
['у меня тоже для тебя кое - что есть .']
И что же?
['у меня тоже для тебя кое - что есть .']
Плохой ответ
['извини , пожалуйста .']
Чем занимаешься?
['нет , не надо .']
Почему сразу не надо?
['у меня тоже для тебя кое - что есть .']
Ты повторяешься
['я пойду с тобой .']
Не ходи со мной
['боже .']
Не поминай в суе
['я пойду с тобой .']
Опять...
['у меня тоже для тебя кое - что есть .']
Ты так себе бот
['что ?']
Выход
['действуйте как чемпионы , будьте чемпионами !']
выход
['футбол это серьёзно .']
